In [16]:
import random
import numpy as np
import os

print("testing...")

testing...


In [73]:
# Consider discretizing the labels so the arrays can be numpy
# DATASET.PARTITIONS FORMAT: dataset.partitions[fold][example][attribute]
'''
self.line_count - total # of examples
self.partition_count - total # of partitions
self.total_example_count - total # of examples
self.attribute_count - total # of attributes for each example
'''

class dataset:
    def __init__(self, data_path, label_location, processed_flag):
        if (processed_flag == False):
            # Separating the .data file into lines, and shuffling the lines
            with open(data_path, 'r') as file:
                lines = file.readlines()
            random.shuffle(lines)
            self.data_lines = lines
            #print(f"Data Lines Array Unprocessed: {self.data_lines}")

            # Deliminate strings into lists
            for i in range(len(self.data_lines)):
                self.data_lines[i] = self.data_lines[i].strip()
                self.data_lines[i] = self.data_lines[i].split(',')

            # If the labels are in the first location, move them to the last location in the line
            #print(f"Data Lines Array Deliminated: {self.data_lines}")
            if (label_location == 'first'):
                for i in range(len(self.data_lines)):
                    first_item = self.data_lines[i].pop(0)
                    self.data_lines[i].append(first_item)
                #print(f"Data Lines Array W/ Indices Swapped: {self.data_lines}")

            # Folding the data...
            self.line_count = len(self.data_lines)
            partition_size = self.line_count//10
            partition_remainder = self.line_count%10
            self.partitions = [self.data_lines[i * partition_size:(i + 1) * partition_size] for i in range(10)] # Separating the lines list into 10 lists of lines
            if (partition_remainder != 0):
                for i in range(partition_remainder):
                    # MIGHT NEED TO ADD EXTRA INDICE DIMENSION WHEN DATA IS DELIMINATED PRIOR
                    self.partitions[:][i].append(self.data_lines[-(i+1)])
        else:
            #extract processed data
            self.partitions = []
            with open(data_path, 'r') as file:  #open csv file of processed data
                for line in file:
                    line = line.strip() #strip each line of any white space or newlines
                    examples = line.split(";") #Split each line into examples by each semicolon
                    attributes = [list(map(str, item.split(","))) for item in examples] #seperate each attribute and remove commas
                    self.partitions.append(attributes) # put into 3d array

        # General dataset information setters
        self.partition_count = 10
        #self.total_example_count = self.line_count
        self.attribute_count = len(self.partitions[0][0])

    def save(self, save_file_name, save_folder):
        # Saves the data based on our convention: Each line is a partition, semicolons separate examples, commas separate attributes/labels
        folder_path = os.path.expanduser(f"{save_folder}/processed_data")  
        os.makedirs(folder_path, exist_ok=True)
        #get/create the path to the folder that the file should be saved to
        file_path = os.path.join(folder_path, save_file_name)
        #create the file path
        with open(f"{file_path}.csv", "w") as file:
            #open a csv file in the desired location
            for line in self.partitions:
                partition_lines = ";".join([",".join(map(str, sub_array)) for sub_array in line])
                #for each partition, join each example by a semi colon and each attribute by a comma
                file.write(partition_lines + "\n")
                #write each partition into the file with each 
        print(f"CSV file saved to {file_path}")
        return

    def imputate(self, voter_bool):
        # Replaces question marks in a dataset with a random value between the min/max of an attribute value
        # Breast cancer has a range of 1-10 for the attribute that is missing values
        # voter_bool indicates whether you are working with the voter dataset, where attributes are strings of 'y' or 'n' instead of numbers
        voter_options = ['y', 'n']
        
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                for attribute in range(len(self.partitions[partition][example])):
                    # if this statement is entered that means there is a missing piece of attribute data, so imputation needs to occur at this location
                    if (self.partitions[partition][example][attribute] == '?'):
                        # This will be the imputation method using 'y' or 'n'
                        if (voter_bool == True):
                            self.partitions[partition][example][attribute] = random.choice(voter_options)
                        # This will be the imputation method using range 1-10
                        else:
                            self.partitions[partition][example][attribute] = str(random.randint(1,10))



    def discretize(self, divisions):
        '''
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                for attribute in range((len(self.partitions[partition][example])-1)):

        attribute_extrema = []
        # Takes a continous attribute and "bins" the attribute into discrete groups. May need to create a dictionary
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                for attribute in range((len(self.partitions[partition][example])-1)):
                    if (self.partitions[partition][example][attribute] > attribute_extrema[attribute])
        '''

        return

    def add_noise(self):
        #selects 10% of the features at random and shuffles the values within each feature, thus introducing noise into the data
        num_to_shuffle = max(1, int(round(0.1 * (self.attribute_count- 1)))) #get the numbr of attributes to shuffle based on 10% of features bing shuffled
        print(f"{self.attribute_count}")
        attribute_noise = random.sample(range(self.attribute_count - 1), num_to_shuffle)# randomly choose an attribute to shuffle
        print(f"Noise Index: {attribute_noise}")
        print(f"No Noise: {self.partitions}" + "\n")
        for attribute_indice in attribute_noise: #loop through each attribute to shuffle in each example and partition
            attributes_values = []
            for partition in self.partitions:
                for example in partition:
                    attributes_values.append(example[attribute_indice]) #store attribute values in a list
            
            random.shuffle(attributes_values) #shuffle selected attribute values


            i = 0
            for partition in self.partitions:  #loop through each partition and example
                    for example in partition:
                        example[attribute_indice] = attributes_values[i] #put the attribute value back into the data after being shuffled
                        i += 1
        print(f"With Noise: {self.partitions}")
     
        
        return

    
        


In [84]:
# dataset instantiation
iris_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/iris.data', 'last', False)

In [85]:
# Constructor Testing

print(f"Folded Array: {iris_dataset.partitions}")
print(f"Data first layer length (Should be length 10): {len(iris_dataset.partitions)}")
#print(f"Data second layer length (Should be {iris_dataset.line_count}): {len(iris_dataset.partitions[1])}")
#print(f"Data third layer length (Should be # of items in a line): {len(iris_dataset.partitions[0][0])}")

total_line_count = 0
for i, partition in enumerate(iris_dataset.partitions):
    print(f'Fold {i+1} Length: {len(partition)}')
    total_line_count += len(partition)
print(f"Number of Examples: {total_line_count}")

#print(iris_dataset.partitions[0][0][2])
print(iris_dataset.attribute_count)

Folded Array: [[['6.5', '3.0', '5.5', '1.8', 'Iris-virginica'], ['6.4', '2.9', '4.3', '1.3', 'Iris-versicolor'], ['5.5', '2.6', '4.4', '1.2', 'Iris-versicolor'], ['5.6', '3.0', '4.1', '1.3', 'Iris-versicolor'], ['5.1', '2.5', '3.0', '1.1', 'Iris-versicolor'], ['6.3', '3.4', '5.6', '2.4', 'Iris-virginica'], ['6.6', '2.9', '4.6', '1.3', 'Iris-versicolor'], ['6.3', '2.7', '4.9', '1.8', 'Iris-virginica'], ['6.1', '3.0', '4.9', '1.8', 'Iris-virginica'], ['5.7', '2.8', '4.5', '1.3', 'Iris-versicolor'], ['6.7', '3.1', '5.6', '2.4', 'Iris-virginica'], ['4.4', '3.2', '1.3', '0.2', 'Iris-setosa'], ['5.0', '2.3', '3.3', '1.0', 'Iris-versicolor'], ['4.7', '3.2', '1.6', '0.2', 'Iris-setosa'], ['5.2', '3.5', '1.5', '0.2', 'Iris-setosa'], ['5.1', '3.8', '1.6', '0.2', 'Iris-setosa']], [['5.0', '3.5', '1.6', '0.6', 'Iris-setosa'], ['4.8', '3.0', '1.4', '0.1', 'Iris-setosa'], ['7.6', '3.0', '6.6', '2.1', 'Iris-virginica'], ['6.7', '3.3', '5.7', '2.5', 'Iris-virginica'], ['5.1', '3.5', '1.4', '0.3', 'Iri

In [86]:
# .save() testing
iris_dataset = dataset("/home/carlos/Machine_Learning_Practice/datasets/iris.data", 'last', False)
save_test = "save_test"
iris_dataset.save(save_test, "/home/carlos/Machine_Learning_Practice")

CSV file saved to /home/carlos/Machine_Learning_Practice/processed_data/save_test


In [87]:
# .extract() testing
iris_dataset = dataset('/home/carlos/Machine_Learning_Practice/processed_data/save_test.csv', 'last', True)

In [88]:
# .imputate() testing
cancer_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/breast-cancer-wisconsin.data', 'last', False)
print(f"Partition size (estimate): {len(cancer_dataset.partitions[0])}")
print(f"Cancer Dataset w/o imputation: {cancer_dataset.partitions}")
cancer_dataset.imputate(False)
print(f"Cancer Dataset w/  imputation: {cancer_dataset.partitions}")

print()

votes_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/house-votes-84.data', 'first', False)
print(f"voter Dataset w/o imputation: {votes_dataset.partitions}")
votes_dataset.imputate(True)
print(f"voter Dataset w/  imputation: {votes_dataset.partitions}")



Partition size (estimate): 70
Cancer Dataset w/o imputation: [[['1049815', '4', '1', '1', '1', '2', '1', '3', '1', '1', '2'], ['1243256', '10', '4', '3', '2', '3', '10', '5', '3', '2', '4'], ['1369821', '10', '10', '10', '10', '5', '10', '10', '10', '7', '4'], ['1313982', '4', '3', '1', '1', '2', '1', '4', '8', '1', '2'], ['508234', '7', '4', '5', '10', '2', '10', '3', '8', '2', '4'], ['1152331', '4', '1', '1', '1', '2', '1', '3', '1', '1', '2'], ['1189286', '10', '10', '8', '6', '4', '5', '8', '10', '1', '4'], ['1218105', '5', '10', '10', '9', '6', '10', '7', '10', '5', '4'], ['529329', '10', '10', '10', '10', '10', '10', '4', '10', '10', '4'], ['1175937', '5', '4', '6', '7', '9', '7', '8', '10', '1', '4'], ['303213', '10', '4', '4', '10', '6', '10', '5', '5', '1', '4'], ['1299596', '2', '1', '1', '1', '2', '1', '1', '1', '1', '2'], ['466906', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2'], ['1350423', '5', '10', '10', '8', '5', '5', '7', '10', '1', '4'], ['1311875', '5', '1', '2',

In [89]:
# .discretize() testing


In [77]:
# .add_noise() testing
glass_data = dataset(r"C:\Users\david\OneDrive\Desktop\CSCI_447\Datasets\glass.data","last", False)
glass_data.add_noise()

voter_data = dataset(r"C:\Users\david\OneDrive\Desktop\CSCI_447\Datasets\house-votes-84.data", "first", False)
voter_data.imputate(True)
print("\n")
voter_data.add_noise()

11
Noise Index: [0]
No Noise: [[['39', '1.52213', '14.21', '3.82', '0.47', '71.77', '0.11', '9.57', '0.00', '0.00', '1'], ['164', '1.51514', '14.01', '2.68', '3.50', '69.89', '1.68', '5.87', '2.20', '0.00', '5'], ['159', '1.51776', '13.53', '3.41', '1.52', '72.04', '0.58', '8.79', '0.00', '0.00', '3'], ['150', '1.51643', '12.16', '3.52', '1.35', '72.89', '0.57', '8.53', '0.00', '0.00', '3'], ['75', '1.51596', '13.02', '3.56', '1.54', '73.11', '0.72', '7.90', '0.00', '0.00', '2'], ['45', '1.51786', '12.73', '3.43', '1.19', '72.95', '0.62', '8.76', '0.00', '0.30', '1'], ['188', '1.52315', '13.44', '3.34', '1.23', '72.38', '0.60', '8.83', '0.00', '0.00', '7'], ['67', '1.52152', '13.05', '3.65', '0.87', '72.22', '0.19', '9.85', '0.00', '0.17', '1'], ['72', '1.51848', '13.64', '3.87', '1.27', '71.96', '0.54', '8.32', '0.00', '0.32', '2'], ['85', '1.51409', '14.25', '3.09', '2.08', '72.28', '1.10', '7.08', '0.00', '0.00', '2'], ['15', '1.51763', '12.61', '3.59', '1.31', '73.29', '0.58', '8.5